In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
import re
import time
from datetime import datetime
import os
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import paramiko


user_name='trishala/'

ssh_host = '10.120.1.50'
ssh_username = 'research-user'
localhost = '127.0.0.1'

# prod
ezcapc_user = 'read_only'
ezcapc_pwd = 'WHC63fKyjudfSz2U'
ezcapc_host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ezcapc_db_name = 'CAPC_APIGATEWAY'
env='production'
paas_user = 'read_only'
paas_pwd = 'eUN2cORh1sCRG998'
paas_host = 'paas-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
paas_db_name = 'FACILITY_SERVICE'

# # stag
# ezcapc_user = 'read_only'
# ezcapc_pwd = '3j4A3PaQc8BV050R'
# ezcapc_host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# ezcapc_db_name = 'CAPC_APIGATEWAY'
# env='staging'
# paas_user = 'read_only'
# paas_pwd = 's9a843QvgkPje5SD'
# paas_host = 'paas-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# paas_db_name = 'METADATA_SERVICE'



# ezcapc_db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(ezcapc_user,ezcapc_pwd,ezcapc_host,ezcapc_db_name)
# ezcapc_db_connection = create_engine(ezcapc_db_connection_str)

# paas_db_connection_str = 'mysql+pymysql://{}:{}@{}'.format(paas_user,paas_pwd,paas_host)
# paas_db_connection = create_engine(paas_db_connection_str)

In [2]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global ezcapc_tunnel
    global paas_tunnel
    ezcapc_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (ezcapc_host, 3306)
    )
    paas_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (paas_host, 3306)
    )
    ezcapc_tunnel.start()
    paas_tunnel.start()
    print(ezcapc_tunnel.is_active)
    print(paas_tunnel.is_active)

In [3]:
def mysql_connect():
    global ezcapc_connection
    global paas_connection
    ezcapc_connection = pymysql.connect(
        host=localhost,
        user=ezcapc_user,
        passwd=ezcapc_pwd,
        database=ezcapc_db_name,
        port=ezcapc_tunnel.local_bind_port
    )

    paas_connection = pymysql.connect(
        host=localhost,
        user=paas_user,
        passwd=paas_pwd,
        database=paas_db_name,
        port=paas_tunnel.local_bind_port
    )

In [4]:
open_ssh_tunnel()
mysql_connect()

True
True


In [40]:
# please change esm.last_coding_date & am.facility_id = '314' in the query as per requirement
# am.facility_id is prod_id from go-live sheet
query = '''
SELECT 
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
    eandm.code AS final_code,
    CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
    GROUP_CONCAT(DISTINCT eandm.code) AS final,
    GROUP_CONCAT(DISTINCT dc.code) AS suggested,
    -- am.account_number AS account_number,
    facility_id,
    client_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        AND dm.is_physician_document = 1
        AND dm.is_active = 1
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
        AND dc.is_active = 1
        AND dc.code_category = 'EANDM'
        INNER JOIN
    encounter_mst em ON em.account_id = dm.account_id
        AND em.service_date = dm.service_date
        AND em.physician_id = dm.physician_id
        AND em.is_active = 1
        INNER JOIN
    encounter_status_map esm ON esm.encounter_id = em.id
        AND esm.is_active = 1
        AND esm.encounter_status_id IN (9 , 17)
        INNER JOIN
    eandm_code eandm ON eandm.encounter_id = em.id
        AND eandm.process_id = esm.process_id
        AND eandm.user_id = esm.user_id
        AND eandm.is_active = 1
	
WHERE
dpd.id = (SELECT 
            id
        FROM
            document_processing_detail
        WHERE
            document_id = dm.id
        ORDER BY created_date DESC
        LIMIT 1)
        AND am.facility_id = '325'
        AND am.client_id = '15'
        AND esm.id = (SELECT 
            id
        FROM
            encounter_status_map
        WHERE
            encounter_id = em.id
                AND encounter_status_id IN (9 , 17)
        ORDER BY last_coding_date DESC
        LIMIT 1)
        AND am.is_active = 1
        AND CONVERT_TZ(esm.last_coding_date,
         'UTC',
         'US/Central') <= '2023-09-15'
        AND CONVERT_TZ(esm.last_coding_date,
         'UTC',
         'US/Central') >= '2023-09-01'

GROUP BY am.account_number , em.physician_id , em.service_date , dm.id
'''

        # AND CONVERT_TZ(esm.last_coding_date,
        #     'UTC',
        #     'US/Central') >= '2023-08-19'
        # AND CONVERT_TZ(esm.last_coding_date,
        #     'UTC',
        #     'US/Central') >= '2023-08-01'
        
        
        # AND CONVERT_TZ(em.service_date,
        #     'UTC',
        #     'US/Central') > '2023-03-15'
        
        #     -- am.account_number = 8594135
        
            #     AND CONVERT_TZ(em.service_date,
            # 'UTC',
            # 'US/Central') > '2023-07-01'

In [41]:
df = pd.read_sql(query, con=ezcapc_connection)

In [42]:
df

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,encounter_id,suggested_code,DPD_ID,filePath,final_code,service_date,final,suggested,facility_id,client_id
0,89577462,ezcapc/SCP/data/3400/nlp/cdp-xml/20230705/1067...,ezcapc/SCP/data/3400/nlp/ner/20230705/10674073...,325,2023-07-05 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230705/1067...,ezcapc/SCP/data/3400/preprocessing/20230705/10...,ezcapc/SCP/data/3400/preprocessing/20230705/10...,SCP/data/3400/document/20230705/647671210.pdf,274266,...,8358844,99285,10752848,SCP/data/3400/document/20230705/647671210_2023...,99285,2023-07-05,99285,99285,325,15
1,89732696,ezcapc/SCP/data/3400/nlp/cdp-xml/20230715/1119...,ezcapc/SCP/data/3400/nlp/ner/20230715/11196104...,325,2023-07-15 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230715/1119...,ezcapc/SCP/data/3400/preprocessing/20230715/11...,ezcapc/SCP/data/3400/preprocessing/20230715/11...,SCP/data/3400/document/20230715/650479632.pdf,274266,...,8735105,99284,11273509,SCP/data/3400/document/20230715/650479632_2023...,99285,2023-07-15,"99284,99285",99284,325,15
2,89732696,ezcapc/SCP/data/3400/nlp/cdp-xml/20230715/1277...,ezcapc/SCP/data/3400/nlp/ner/20230715/12771043...,325,2023-07-15 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230715/1277...,ezcapc/SCP/data/3400/preprocessing/20230715/12...,ezcapc/SCP/data/3400/preprocessing/20230715/12...,SCP/data/3400/document/20230715/659204393.pdf,274266,...,8735105,99285,12845513,SCP/data/3400/document/20230715/659204393_2023...,99285,2023-07-15,"99284,99285",99285,325,15
3,89766167,ezcapc/SCP/data/3400/nlp/cdp-xml/20230717/1188...,ezcapc/SCP/data/3400/nlp/ner/20230717/11882042...,325,2023-07-17 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230717/1188...,ezcapc/SCP/data/3400/preprocessing/20230717/11...,ezcapc/SCP/data/3400/preprocessing/20230717/11...,SCP/data/3400/document/20230717/653851077.pdf,274266,...,9214661,99285,11958924,SCP/data/3400/document/20230717/653851077_2023...,99285,2023-07-17,99285,99285,325,15
4,89782684,ezcapc/SCP/data/3400/nlp/cdp-xml/20230718/1119...,ezcapc/SCP/data/3400/nlp/ner/20230718/11197034...,325,2023-07-18 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230718/1119...,ezcapc/SCP/data/3400/preprocessing/20230718/11...,ezcapc/SCP/data/3400/preprocessing/20230718/11...,SCP/data/3400/document/20230718/650485632.pdf,274266,...,8735835,99283,11274399,SCP/data/3400/document/20230718/650485632_2023...,99285,2023-07-18,99285,99283,325,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,90546605,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1264...,ezcapc/SCP/data/3400/nlp/ner/20230902/12642717...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1264...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/657980464.pdf,274266,...,9727920,99284,12717614,SCP/data/3400/document/20230902/657980464_2023...,99284,2023-09-02,99284,99284,325,15
770,90546606,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1264...,ezcapc/SCP/data/3400/nlp/ner/20230902/12642726...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1264...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/657980543.pdf,274266,...,9727924,99284,12717620,SCP/data/3400/document/20230902/657980543_2023...,99284,2023-09-02,99284,99284,325,15
771,90546607,ezcapc/SCP/data/3400/nlp/cdp-xml/20230902/1264...,ezcapc/SCP/data/3400/nlp/ner/20230902/12642714...,325,2023-09-02 05:00:00,ezcapc/SCP/data/3400/nlp/pcs-xml/20230902/1264...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,ezcapc/SCP/data/3400/preprocessing/20230902/12...,SCP/data/3400/document/20230902/657980394.pdf,274266,...,9727916,99284,12717610,SCP/data/3400/document/20230902/657980394_2023...,99284,20

In [43]:
df = df[['service_date', 'account_number', 'suggested', 'final']]

In [44]:
df.suggested.value_counts()

99284    527
99285    104
99283     93
99282     33
99291     17
Name: suggested, dtype: int64

In [45]:
# df.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/neighbourhood_testing/ACCURACY_MONITORING/493937_BILLED.xlsx', index=False)

In [46]:
df[df.suggested.str.contains('99281')]

,service_date,service_date,account_number,suggested,final


In [47]:
# check for nan in final and suggested columns

In [48]:
df

,service_date,service_date,account_number,suggested,final
0,2023-07-05 05:00:00,2023-07-05,89577462,99285,99285
1,2023-07-15 05:00:00,2023-07-15,89732696,99284,"99284,99285"
2,2023-07-15 05:00:00,2023-07-15,89732696,99285,"99284,99285"
3,2023-07-17 05:00:00,2023-07-17,89766167,99285,99285
4,2023-07-18 05:00:00,2023-07-18,89782684,99283,99285
...,...,...,...,...,...
769,2023-09-02 05:00:00,2023-09-02,90546605,99284,99284
770,2023-09-02 05:00:00,2023-09-02,90546606,99284,99284
771,2023-09-02 05:00:00,2023-09-02,90546607,99284,99284
772,2023-09-03 05:00:00,2023-09-03,90569723,99285,99285


In [49]:
df = df[~df.final.str.contains(',')]

In [50]:
def get_counts(x):
    
    suggested = str(x['suggested'])
    final = str(x['final'])
    
    suggested_set = set(suggested.split(','))   
    final_set = set(final.split(','))
    
    accepted = len(suggested_set.intersection(final_set))
    rejected = len(suggested_set - final_set)
    added = len(final_set - suggested_set)
    if (rejected >= 1 or added >= 1):
        print('rejected',rejected)
        print('added',added)
        print('suggested_set',suggested_set,'\t final_set',final_set)
        print('accountNumber', str(x['account_number']))
            
    return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [51]:
count_df =  df.apply(get_counts, axis=1)

rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99285'}
accountNumber 89782684
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99285'}
accountNumber 89782695
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99284'}
accountNumber 89877913
rejected 1
added 1
suggested_set {'99285'} 	 final_set {'99291'}
accountNumber 89877923
rejected 1
added 1
suggested_set {'99284'} 	 final_set {'99285'}
accountNumber 89938251
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99284'}
accountNumber 89938271
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99284'}
accountNumber 89954476
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99285'}
accountNumber 89954488
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99285'}
accountNumber 89970070
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99284'}
accountNumber 89970073
rejected 1
added 1
suggested_set {'99283'} 	 final_set {'99284'}
accountNumber 89985824
rejected 1
added 1
suggested_set

In [52]:
count_df

,accepted,rejected,added
0,1,0,0
3,1,0,0
4,0,1,1
5,1,0,0
6,0,1,1
...,...,...,...
769,1,0,0
770,1,0,0
771,1,0,0
772,1,0,0


In [53]:
count_df.accepted.sum(), count_df.rejected.sum(), count_df.added.sum()

(439, 200, 200)

In [54]:
precision = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.rejected.sum())

In [55]:
recall = count_df.accepted.sum() / (count_df.accepted.sum() + count_df.added.sum())

In [56]:
f1_score = (2 * precision * recall) / (recall + precision)

In [57]:
precision, recall, f1_score

(0.6870109546165885, 0.6870109546165885, 0.6870109546165885)